### 0)
Relevant imports for this solution

In [ ]:
import os
import random
#helpful ml modules
import matplotlib.pyplot as plt
import numpy as np
import cv2
#Tensorflow and keras
import tensorflow as tf
from tensorflow import keras
#create sequential models
from tensorflow.keras.models import Model
#import the layers
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, RandomBrightness

### 1)
- image = cv2.imread('your_image_path.jpg')
- (240, 320, 3)
- Early Assumption, this model has 3 classes and a binary output  
- The shape is the inputinformation for the network

### 2a)
This model has 2 different outputs so the dataset needs 2 labels. Additionally the dataset is created with the file name from a single folder. The files are named after their color and their state (fail or not fail. Example: blue_45 is of class blue and state not fail, blue_fail_37 is of class blue and state fail). Same method for training and validation set.

In [ ]:
# trainingset
# Initialize empty lists for data and labels
x_train = []
y_train_class = []
y_train_score = []

# Define the path to your training dataset directory
training_directory = 'Datasets/train_folder'

# List all Files in the training dataset folder and shuffle them
files = os.listdir(training_directory)
random.shuffle(files)

# Iterate through the files and load each image into the x variable
# Determin their y labels via their filename
for x in range(0,len(files)):

    img = cv2.imread(os.path.join(train_folder, files[x]))
    x_train.append(img)

    # Determine the class
    if "blue" in files[x]:
        y_train_class.append(0)
    elif "red" in files[x]:
        y_train_class.append(1)
    elif "white" in files[x]:
        y_train_class.append(2)
    # Determine the state
    if "fail" in files[x]:
        y_train_score.append(False)
    else:
        y_train_score.append(True)

# Cast the lists to numpy arrays for better processing options
x_train = np.array(x_train)
y_train_class = np.array(y_train_class)
y_train_score = np.array(y_train_score)

# Cast the index to a tuple where the number at the index is 1 and the others are 0
# --> index of class blue is 0 --> translates to {1,0,0}
# --> index of class red is 1 --> translates to {0,1,0}
# --> index of class white is 2 --> translates to {0,0,1}
y_train_class = tf.keras.utils.to_categorical(y_train_class, num_classes=num_classes)

# Print the shape of your dataset to check for the correct number of examples and size of the images
print("x_train shape", x_train.shape)
print("y_train_class shape", y_train_class.shape)
print("y_train_score shape", y_train_score.shape)

### 3)
This is just a suggestion, try around to get a good model

In [ ]:
# Building the model

# Convolutional Layer parameters
num_filters = 100
filter_size = (3,3)
stride = 1

# Pooling parameter
pool_size = (2,2)
# Density Layer parameters
num_dense_units = 100

# Additional parameters
brightnessfactor = 0,25
dropout_rate = 0,1
shape = (240, 320, 3)
num_classes = 3

# Define input layer
input_layer = Input(shape=shape)
x = input_layer

# Augmentation Layer
x = RandomBrightness(brightnessfactor)(x)

# Convolutional layers
x = Conv2D(num_filters, filter_size, strides=stride, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(dropout_rate)(x)

x = MaxPooling2D(pool_size=pool_size)(x)

x = Conv2D(num_filters/2, filter_size, strides=stride, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(dropout_rate)(x)

x = MaxPooling2D(pool_size=pool_size)(x)

x = Flatten()(x)

x = Dense(num_dense_units, activation='relu')(x)
x = Dropout(dropout_rate)(x)

# Output layers
class_output = Dense(num_classes, activation='softmax', name='class_output')(x)
score_output = Dense(1, activation='sigmoid', name='score_output')(x)

# Define the model
model = Model(inputs=input_layer, outputs=[class_output, score_output])

model.summary()

In [ ]:
# Compile your model
# Extra parameter because there are 2 outputs with separate losses
loss_weights = {'class_output': 0.175, 'score_output': 0.825}

model.compile(optimizer='adam',
                      loss= {
                            'class_output': tf.keras.losses.CategoricalCrossentropy(),
                            'score_output': tf.keras.losses.BinaryCrossentropy()
                            },
                      metrics=['accuracy'],
                      loss_weights = loss_weights)

### 3a)

In [ ]:
# Define early stopping function to stop the training automatically
# when it does not improve the model after  number of epochs defined by patience
patience = 15
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

# Training of the model. The Dataset here consists of 3 elements because of the model architecture.
num_epochs = 200 # Can be high because of early stopping
batch_size = 32
model.fit(x_train, {'class_output': y_train_class, 'score_output': y_train_score},
                            epochs=num_epochs,
                            batch_size=batch_size,
                            validation_data=(x_val, {'class_output': y_val_class, 'score_output': y_val_score}),
                            callbacks=[early_stopping])


### 3b)
Optimize Options:
- Dataaugmentation -> Extra layers for augmentation (Example: Randombrightness)
- Regularization -> Dropout or L2
- Train longer -> more epochs
- increase the complexity of the network -> more Layers

In [ ]:
model.evaluate(x_val,{'class_output': y_val_class, 'score_output': y_val_score}, batch_size = batch_size)

### 4)

In [ ]:
# Keras Layers for Augmentation
x = RandomBrightness(brightnessfactor)(x)
# Layer for Regularization
x = Dropout(dropout_rate)(x)